# Training CycleGANs
This is the version meant to be used on the pods, and this will train Monet. 

In [1]:
%matplotlib notebook
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import time
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
def getDataset(path,image_size=(256,256)):
    transform = tv.transforms.Compose([
        tv.transforms.Resize(image_size,Image.LANCZOS),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])
    dataset = tv.datasets.ImageFolder(path,transform) #what I downloaded
    return dataset
    

In [4]:
landscapes_train= getDataset('landscape_photos/train/')
landscapes_test= getDataset('landscape_photos/test/')

If we are using the dataset hosted on the servers

In [5]:
#landscapes = getDataset('/datasets/ee285f-public/flickr_landscape')
#train_size = int(0.8 * len(landscapes))
#test_size = len(landscapes) - train_size
#landscapes_train, landscapes_test = torch.utils.data.random_split(landscapes, [train_size, test_size])

In [6]:
!cp /datasets/ee285f-public/wikiart/Artist/artist_train ./artist_train.csv
!cp /datasets/ee285f-public/wikiart/Artist/artist_val ./artist_test.csv

In [7]:
class WikiartArtistDataset(td.Dataset):
    def __init__(self, root_dir, mode="train", image_size=(256, 256)):
        super(WikiartArtistDataset, self).__init__()
        self.image_size = image_size
        self.mode = mode
        csv = pd.read_csv("./artist_%s.csv" % mode,header=None)
        csv = csv.iloc[1:,]
        csv.index = range(len(csv))
        self.csv = csv
        self.data = self.csv[0].tolist()
        self.artists = self.csv[2].tolist()
        self.images_dir = os.path.join(root_dir)
    def __len__(self):
        return len(self.data)
    def __repr__(self):
        return "WikiartDataset(mode={}, image_size={})". \
            format(self.mode, self.image_size)
    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, \
                                self.data[idx])
        img = Image.open(img_path).convert('RGB')
        transform = tv.transforms.Compose([
            tv.transforms.Resize(self.image_size),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
            ])
        x = transform(img)
        d = self.artists[idx]
        return x, d
    def number_of_classes(self):
        return len(set(self.artists))
    def get_name(self,idx):
        return self.data[idx]
    

In [8]:
image_root_dir='/datasets/ee285f-public/wikiart/wikiart/'
art_train = WikiartArtistDataset(image_root_dir,mode='train')
art_test = WikiartArtistDataset(image_root_dir,mode='test')

In [9]:
!cp /datasets/ee285f-public/wikiart/Artist/artist_class ./artist_class.csv
artist_class = pd.read_csv('./artist_class.csv', header=None)

In [10]:
artist_class.values # choose an artist id here 

array([['0 Albrecht_Durer'],
       ['1 Boris_Kustodiev'],
       ['2 Camille_Pissarro'],
       ['3 Childe_Hassam'],
       ['4 Claude_Monet'],
       ['5 Edgar_Degas'],
       ['6 Eugene_Boudin'],
       ['7 Gustave_Dore'],
       ['8 Ilya_Repin'],
       ['9 Ivan_Aivazovsky'],
       ['10 Ivan_Shishkin'],
       ['11 John_Singer_Sargent'],
       ['12 Marc_Chagall'],
       ['13 Martiros_Saryan'],
       ['14 Nicholas_Roerich'],
       ['15 Pablo_Picasso'],
       ['16 Paul_Cezanne'],
       ['17 Pierre_Auguste_Renoir'],
       ['18 Pyotr_Konchalovsky'],
       ['19 Raphael_Kirchner'],
       ['20 Rembrandt'],
       ['21 Salvador_Dali'],
       ['22 Vincent_van_Gogh']], dtype=object)

In [11]:
#artist_class
class SpecificArtistDataset(td.Dataset):
    def __init__(self,root_dir, artist_id, wikiart, image_size=(256, 256)):
        super(SpecificArtistDataset, self).__init__()
        self.image_size = image_size
        csv = wikiart
        csv = wikiart[wikiart[2] == artist_id]
        csv.index = range(len(csv))
        self.csv = csv
        self.data = self.csv[0].tolist()
        self.artists = self.csv[2].tolist()
        self.images_dir = os.path.join(root_dir)
    def __repr__(self):
        return "SpecificArtistDataset(image_size={})". \
            format(self.image_size)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, \
                                self.data[idx])
        img = Image.open(img_path).convert('RGB')
        transform = tv.transforms.Compose([
            tv.transforms.Resize(self.image_size,Image.LANCZOS),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
            ])
        x = transform(img)
        return x,artist_id
    def get_name(self,idx):
        return self.data[idx]
    

In [12]:
artist_id ='4' # Monet

In [13]:
artist_train = SpecificArtistDataset(image_root_dir,artist_id,art_train.csv) ## change the number for different artist
artist_test = SpecificArtistDataset(image_root_dir,artist_id,art_test.csv)

If we are using dataset from the author like for Ukiyo-e and Van Gogh we just place the data into train and test subfolders and use getDataset()

In [14]:
def myimshow(image, ax=plt):
    image = image.to('cpu').numpy()
    image = np.moveaxis(image, [0, 1, 2], [2, 0, 1])
    image = (image + 1) / 2
    image[image < 0] = 0
    image[image > 1] = 1
    h = ax.imshow(image)
    ax.axis('off')
    return h

In [15]:
#myimshow(artist_train[1][0])

In [16]:
#artist_train.get_name(1)


## Params 

From paper:

Generator
 9 residual blocks consists
c7s1-64,d128,d256,R256,R256,R256,
R256,R256,R256,R256,R256,R256,u128
u64,c7s1-3

Discriminator:
C64-C128-C256-C512 where 4x4 Convolution-Instance-Norm-LeakyReLU layer with k filters and stride 2. Last layer is followed by a convolution to 1 dimensional. First layer does not have instance norm. leakyrelu(0.2)

In [17]:
lr = 0.0002 # this is what they used in paper and decayed to 0 after 100 epochs
beta1 = 0.5 # beta values can try tuning in future
beta2 = 0.999
lambda_cyc = 10 #from the paper, we tried 7 for the lower lambda experiment
batch_size = 1 # not sure
epochs = 200 #
lambda_iden = 0.5

In [18]:
import network as net #discriminators and generators from renjie in network.py 

## Helper 

In [19]:
#history of generated images
class fake_hist():
    def __init__(self, size):
        self.size = size
        self.count = 0
        self.fakes = []
    def get_fake(self, new_fakes):
        toReturn = []
        for fake in new_fakes:
            image = torch.unsqueeze(fake.data, 0)
            if self.count < self.size:
                self.count+=1
                self.fakes.append(image)
                toReturn.append(image)
            else:
                #either get random image from the history or return this image
                if random.random() > 0.5:
                    index = random.randint(0, self.size - 1) 
                    toReturn.append(self.fakes[index].clone())
                    self.fakes[index] = image
                else:
                    toReturn.append(image)
        return torch.cat(toReturn, 0) 

In [20]:
#add noise
def addNoise(img, mean=0, std=0.1):
    noise = Variable(img.data.new(img.size()).normal_(mean, std))
    return img + noise

## Training class

In [119]:
# some code taken from nntools
class TrainCycleGAN(object):
    def __init__(self, Dx,Dy,Gxy,Gyx, artist_train,landscapes_train,
                 output_dir,device,gen_optim,dis_optim, batch_size=1,
                 epochs=200,lambda_cyc=10,lambda_iden=0.5,load_epoch='0'):
        self.epochs = epochs
        self.epoch_count = 1
        self.Dx = Dx
        self.Dy = Dy
        self.Gxy = Gxy
        self.Gyx = Gyx
        self.device = device
        self.gen_optim = gen_optim
        self.dis_optim = dis_optim
        self.art_hist = fake_hist(50)
        self.landscape_hist = fake_hist(50)
        self.lambda_cyc = lambda_cyc
        self.lambda_iden = lambda_iden
        self.dx_loss = []
        self.dy_loss = []
        self.g_loss = []
        self.cyc_loss = []
        self.idt_loss = []
        
        self.landscape_loader = td.DataLoader(landscapes_train, batch_size=batch_size,shuffle=True, pin_memory=True)
        self.art_loader = td.DataLoader(artist_train,batch_size=batch_size,shuffle=True, pin_memory=True)
        self.art_itr = iter(self.art_loader)
        self.landscape_itr = iter(self.landscape_loader)
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(output_dir+'models/', exist_ok=True)
        os.makedirs(output_dir+'images/', exist_ok=True)
        self.model_path = os.path.join(output_dir,'models/')
        self.img_path = os.path.join(output_dir,'images/')
        
        #might want to load a specific epoch before training goes bad
        if(load_epoch != '0'):
            if not os.path.isfile(self.model_path+load_epoch+'_state.pt'):
                        raise ValueError('No saved models for given epoch')
            else:
                self.load(load_epoch)
                
    def state_dict(self):
        """Returns the current state of the experiment."""
        return {'Dx': self.Dx.state_dict(),
                'Dy': self.Dy.state_dict(),
                'Gxy': self.Gxy.state_dict(),
                'Gyx': self.Gyx.state_dict(),
                'gen_optim': self.gen_optim.state_dict(),
                'dis_optim': self.dis_optim.state_dict(),
                'Epoch': self.epoch_count}
    def save(self):
            torch.save(self.state_dict(), self.model_path+str(self.epoch_count)+'_state.pt')
            d = {'Dx loss':self.dx_loss,'Dy loss':self.dy_loss, 'G loss':self.g_loss,
                 'Cycle loss':self.cyc_loss, 'Iden loss': self.idt_loss} 
            df = pd.DataFrame(d)
            df.to_csv(output_dir+'losses.csv', index=False)
    def load(self,load_epoch):
        """Loads the experiment from the last checkpoint saved on disk."""
        checkpoint = torch.load(self.model_path+load_epoch+'_state.pt',
                                map_location=self.device)
        self.load_state_dict(checkpoint)
        del checkpoint
        losses = pd.read_csv(output_dir+'losses.csv').iloc[:int((int(load_epoch)/5))]
        self.dx_loss = losses['Dx loss'].tolist()
        self.dy_loss = losses['Dy loss'].tolist()
        self.g_loss = losses['G loss'].tolist()
        self.cyc_loss = losses['Cycle loss'].tolist()
        self.idt_loss = losses['Iden loss'].tolist()

    def load_state_dict(self, checkpoint):
        """Loads the experiment from the input checkpoint."""
        self.Dx.load_state_dict(checkpoint['Dx'])
        self.Dy.load_state_dict(checkpoint['Dy'])
        self.Gxy.load_state_dict(checkpoint['Gxy'])
        self.Gyx.load_state_dict(checkpoint['Gyx'])
        self.Dx.to(self.device)
        self.Dy.to(self.device)
        self.Gxy.to(self.device)
        self.Gyx.to(self.device)
        self.gen_optim.load_state_dict(checkpoint['gen_optim'])
        self.dis_optim.load_state_dict(checkpoint['dis_optim'])
        self.epoch_count= checkpoint['Epoch'] + 1
    def setLR(self):
        if(self.epoch_count > 100):
            new_lr = 0.0002 - 0.0002*(self.epoch_count-100)/100
            if(new_lr)<0:
                new_lr = 0
            for param_group in self.dis_optim.param_groups:
                param_group['lr'] = new_lr
            for param_group in self.gen_optim.param_groups:
                param_group['lr'] = new_lr    
    def train(self,test_art,test_landscape,noise=False,identity=False,limitupdates=False):
        epochs_left = self.epochs - self.epoch_count + 1
        min_amount = min(len(self.art_itr), len(self.landscape_itr))
        remaining_iterations = epochs_left*min_amount
        total = self.epochs* min_amount #total iterations is epoch * size of smallest dataset
        starting = (self.epoch_count-1)*min_amount # how many already completed if loaded
        std_dev = 0.1 - 0.1*(self.epoch_count/200)
        self.setLR()
        f, axes = plt.subplots(2,3, figsize=(9,6))
        epoch_start = time.time()
        
        print(self.epoch_count)
        for i in range(starting+1,total):
            #ran out of images in one of the sets, start over if still have iterations
            if i % min_amount == 0:
                print(time.time() - epoch_start)
                epoch_start = time.time()
                self.art_itr = iter(self.art_loader)
                self.landscape_itr = iter(self.landscape_loader)
                self.epoch_count+=1
                print(self.epoch_count)
                self.setLR()
                std_dev = 0.1 - 0.1*(self.epoch_count/200)
            real_art =self.art_itr.next()[0].to(self.device)
            real_landscape = self.landscape_itr.next()[0].to(self.device)
            fake_art = self.Gyx(real_landscape)
            fake_landscape= self.Gxy(real_art)

            #don't need to do anything with discriminator gradient here
            for param in self.Dx.parameters():
                param.requires_grad = False
            for param in self.Dy.parameters():
                param.requires_grad = False
            #Generator loss
            self.gen_optim.zero_grad()
            recon_landscape = self.Gxy(fake_art)
            recon_art = self.Gyx(fake_landscape)
            cycle_loss = (((real_art - recon_art).abs()).mean()) + (((real_landscape - recon_landscape).abs()).mean())
            #also added noise to input here in one version as seen in report
            Gyx_loss = ((self.Dx(fake_art) - 1)**2).mean() 
            Gxy_loss = ((self.Dy(fake_landscape) - 1)**2).mean()  #want to fool the discriminator so 1 is better
            G_loss = Gxy_loss + Gyx_loss + self.lambda_cyc*cycle_loss
            iden_loss = 0
            if(identity):
                loss_iden_art = ((self.Gyx(real_art) - real_art).abs()).mean()
                loss_iden_landscape = ((self.Gxy(real_landscape) - real_landscape).abs()).mean()
                iden_loss = loss_iden_art + loss_iden_landscape
                G_loss = Gxy_loss + Gyx_loss + self.lambda_cyc*cycle_loss + iden_loss*self.lambda_cyc*self.lambda_iden
            G_loss.backward()
            self.gen_optim.step()

            #Discriminator Loss
            for param in self.Dx.parameters():
                param.requires_grad = True
            for param in self.Dy.parameters():
                param.requires_grad = True
                dis_optim.zero_grad()

            gen_art = self.art_hist.get_fake(fake_art).detach()
            gen_landscape = self.landscape_hist.get_fake(fake_landscape).detach()
            if noise:
                real_art = addNoise(real_art,0,std=std_dev)
                real_landscape = addNoise(real_landscape,0,std=std_dev)
                gen_art = addNoise(gen_art,0,std=std_dev)
                gen_landscape = addNoise(gen_landscape,0,std=std_dev)
            #loss on real images
            Dx_loss_real = ((self.Dx(real_art) - 1)**2).mean() # x is art domain
            Dy_loss_real = ((self.Dy(real_landscape) - 1)**2).mean() # y is landscape domain
            #loss on generated images
            Dx_fake_pred = self.Dx(gen_art);
            Dy_fake_pred = self.Dy(gen_landscape)

            Dx_loss_fake = (Dx_fake_pred**2).mean()
            Dy_loss_fake = (Dy_fake_pred**2).mean()
            #want d to minimize the loss between fake and real, so combine.
            Dx_loss = 0.5*Dx_loss_real+0.5*Dx_loss_fake
            Dy_loss = 0.5*Dy_loss_real+0.5*Dy_loss_fake
            if not limitupdates: # update only when prediction is bad
                Dx_loss.backward()
                Dy_loss.backward()
                self.dis_optim.step()
            elif (Dx_fake_pred.mean() > 0.5 or Dy_fake_pred.mean() > 0.5):
                Dx_loss.backward()
                Dy_loss.backward()
                self.dis_optim.step()

            #update
            if i % min_amount == 0:
                print('Iteration [{:5d}/{:5d}] | Dy_loss: {:6.4f} | Dx_loss: {:6.4f} | '
                      'g_loss: {:6.4f}'.format(
                        i, total, Dy_loss.data.item(),
                        Dx_loss.data.item(), G_loss.data.item()))

            #every 5 epochs
            if i % (int(remaining_iterations/40)) == 0:
                self.Gxy.eval()
                self.Gyx.eval()
                with torch.no_grad():
                    test_fake_art = self.Gyx(test_landscape).detach()
                    test_fake_landscape = self.Gxy(test_art).detach()
                    recon_art= self.Gyx(test_fake_landscape).detach()
                    recon_landscape =  self.Gxy(test_fake_art).detach()
                    myimshow(test_art[0], ax=axes[0][0])
                    myimshow(test_fake_art[0], ax=axes[1][1])
                    myimshow(test_landscape[0], ax=axes[1][0])
                    myimshow(test_fake_landscape[0], ax=axes[0][1])
                    myimshow(recon_landscape[0], ax=axes[1][2])
                    myimshow(recon_art[0], ax=axes[0][2])
                    plt.tight_layout()
                    f.canvas.draw()
                    f.savefig(self.img_path+str(self.epoch_count)+'_test.jpg', bbox_inches='tight')
                self.Gxy.train()
                self.Gyx.train()
                self.dx_loss.append(Dx_loss.data.tolist())
                self.dy_loss.append(Dy_loss.data.tolist())
                self.g_loss.append(G_loss.data.tolist())
                self.cyc_loss.append(cycle_loss.data.tolist())
                if iden_loss:
                    self.idt_loss.append(iden_loss.data.tolist())
                else:
                    self.idt_loss.append(0)
                self.save()
        self.save()
                    
            

In [120]:
#!mkdir cyclegan_monet_base_train
#!mkdir cyclegan_monet_base_train/models

In [121]:
output_dir = 'cyclegan_monet_base_train/'
model_dir = 'cyclegan_monet_base_train/models/'

# Training

## Discriminators

In [122]:
Dx = net.define_D().to(device)
Dy = net.define_D().to(device)
dis_params = list(Dx.parameters()) + list(Dy.parameters())

## Generators

In [123]:
Gxy = net.define_G().to(device)
Gyx = net.define_G().to(device)
gen_params = list(Gxy.parameters()) + list(Gyx.parameters())

Optimizer

In [124]:
dis_optim = torch.optim.Adam(dis_params, lr, [beta1, beta2])
gen_optim = torch.optim.Adam(gen_params, lr, [beta1, beta2])

In [125]:
landscape_loader_test = td.DataLoader(landscapes_test, batch_size=batch_size,shuffle=True, pin_memory=True)
art_loader_test= td.DataLoader(artist_test,batch_size=batch_size,shuffle=True, pin_memory=True)

In [126]:
art_test_itr= iter(art_loader_test)
landscape_test_itr = iter(landscape_loader_test)

In [127]:
test_art = art_test_itr.next()[0].to(device)
test_landscape = landscape_test_itr.next()[0].to(device)

In [128]:
exp = TrainCycleGAN(Dx,Dy,Gxy,Gyx,artist_train,landscapes_train,output_dir,device,gen_optim,dis_optim)

In [ ]:
exp.train(test_art,test_landscape)

In [130]:
test_landscape2 = landscape_test_itr.next()[0].to(device)

In [131]:
test_art2 = art_test_itr.next()[0].to(device)

In [ ]:
f, axes = plt.subplots(2,3,figsize=(9,6))
exp.Gxy.eval()
exp.Gyx.eval()
with torch.no_grad():
    test_fake_art = exp.Gyx(test_landscape2).detach()
    test_fake_landscape = exp.Gxy(test_art2).detach()
    recon_art= exp.Gyx(test_fake_landscape).detach()
    recon_landscape =  exp.Gxy(test_fake_art).detach()
    myimshow(test_art2[0], ax=axes[0][0])
    myimshow(test_fake_art[0], ax=axes[1][1])
    myimshow(test_landscape2[0], ax=axes[1][0])
    myimshow(test_fake_landscape[0], ax=axes[0][1])
    myimshow(recon_landscape[0], ax=axes[1][2])
    myimshow(recon_art[0], ax=axes[0][2])
    f.canvas.draw()
exp.Gxy.train()
Gyx =exp.Gyx.train() # just to get rid of net structure print